# PDB Preparer
This notebook prepares the starting geometries. In this first cell we define the basic parameters.

In [1]:
from os.path import join, exists
from futile.Utils import ensure_dir
from os import system
from os import environ

geomdir = "raw-structures"
tempdir = "temp"
ensure_dir(tempdir)
outdir = "processed-structures"
ensure_dir(outdir)
picdir = "pictures"
ensure_dir(picdir)
protonated = "protonated"
ensure_dir(protonated)
forcefield = "mmff94"
babel_convergence = 1.0
geom = environ.get("SYSTEM", "Mpro-x0305")

## Metadata Loading
First we load in the metadata to know all the details of this system.

In [2]:
from pickle import load
pname = "updated-data.pickle"
with open(pname, "rb") as ifile:
    data = load(ifile)

In [3]:
row = data.loc[data['Crystal ID'] == geom]
site = row["Site"].iloc[0]
link = row["Link"].iloc[0]
smi = row["Compound SMILES"].iloc[0]
msmi = row["Modified Compound SMILES"].iloc[0]

## Adding Hydrogens

We begin with an initial run of pdbfixer which takes care of all the separate chains and occupancies.

In [4]:
def pdbfixer_basic(geom, outname):
    system("pdbfixer " + join(geomdir, geom+".pdb") + 
           " --add-atoms=none --output=" + outname)

outname = join(tempdir, geom+"-fix.pdb")
if not exists(outname):
    pdbfixer_basic(geom, outname)

Now we add hydrogens to the protein and remove the water/DMS/etc. We may need to loop if pdbfixer doesn't put them in good locations.

In [5]:
from os import remove
from BigDFT.IO import write_pdb, read_pdb

def pdbfixer_addh(geom, outname):
    system("pdbfixer " + join(tempdir, geom+"-fix.pdb") + 
           " --add-atoms=hydrogen --output=" + outname)
    
def clean_ligand_and_solution(inname, outname):
    from copy import deepcopy
    with open(inname) as ifile:
        sys = read_pdb(ifile)
    keys = deepcopy(list(sys.keys()))
    for k in keys:
        if "LIG" in k or "DMS" in k or "HOH" in k:
            del sys[k]
    with open(outname, "w") as ofile:
        write_pdb(sys, ofile)
    
def check_forcefield(geomfile, outname):
    system("obenergy -ff " + forcefield + " " + geomfile + 
           " > " + outname)
    
def check_energy():
    outname3 = join(tempdir, geom+"-energy.txt")
    found = False
    if not exists(outname3):
        return False
    with open(outname3) as ifile:
        found = False
        for line in ifile:
            if "TOTAL ENERGY" in line:
                found = True
                break

    return found

finished = check_energy()
while(not finished):
    finished = True
    outname1 = join(tempdir, geom+"-fixh.pdb")
    pdbfixer_addh(geom, outname1)

    inname = join(tempdir, geom+"-fixh.pdb")
    outname2 = join(tempdir, geom+"-cleaned.pdb")
    clean_ligand_and_solution(inname, outname2)

    inname = join(tempdir, geom+"-cleaned.pdb")
    outname3 = join(tempdir, geom+"-energy.txt")
    check_forcefield(inname, outname3)

    # Check for success
    finished = check_energy()
    if not finished:
        print("Repeat: ", geom)
        remove(outname1)
        remove(outname2)
        remove(outname3)

Last we have to remove the hydrogen which has been cleaved when the ligand forms a covalent bond.

In [6]:
from numpy.linalg import norm
from numpy import array, argmin, argsort

iname = join(tempdir, geom+"-cleaned.pdb")
oname = join(tempdir, geom+"-cleaved.pdb")

with open(iname) as ifile:
    sys = read_pdb(ifile)

if "covalent" in site:
    iname = join(tempdir, geom+"-fixh.pdb")
    with open(iname) as ifile:
        sys_with_lig = read_pdb(ifile)
    ligname = [x for x in sys_with_lig.keys() if "LIG" in x][0]

    # First find out which atom in the ligand is closest to the linking residue.
    distlist = []
    for i, at in enumerate(sys_with_lig[ligname]):
        inner = [norm(array(x.get_position()) - array(at.get_position())) for x in sys_with_lig[link]]
        distlist.append(min(inner))

    # Now the second closest atom will the cleaved hydrogen.
    minat = argmin(distlist)
    outer = [norm(array(x.get_position()) - array(sys_with_lig[ligname][minat].get_position())) 
             for x in sys_with_lig[link]]
    idx = argsort(outer)[1]

    # Remove it and write to file
    del sys[link][idx]

if not exists(oname):
    with open(oname, "w") as ofile:
        write_pdb(sys, ofile)

### Ligand Hydrogens

Now write out the ligand without hydrogens including its bonding information.

In [7]:
from BigDFT.Systems import System

iname = join(tempdir, geom+"-fixh.pdb")
oname = join(tempdir, geom+"-justligand.pdb")
oname2 = join(tempdir, geom+"-justligand-babel.pdb")

with open(iname) as ifile:
    sys = read_pdb(ifile)

target = [x for x in sys.keys() if "LIG" in x][0]

lsys = System()
lsys[target] = sys[target]

if not exists(oname):
    with open(oname, 'w') as ofile:
        write_pdb(lsys, ofile)

if not exists(oname2):
    system("obabel -ipdb " + oname + " -h -opdb > " + oname2)

Add everything together into one big system, and reoptimize the hydrogen atoms of the ligand an its covalent link.

In [8]:
def constrained_optimization_xtb(sys):
    from BigDFT.XTBInterop import XTBCalculator
    code = XTBCalculator(verbose=True)
    constr = {}
    for fragid, frag in sys.items():
        constr[fragid] = []
        for at in frag:
            constr[fragid].append(at.sym != "H")
    code.run(sys=sys, name=geom, run_dir=tempdir, opt=True,
             uhf=2, constraints=constr)
    return code.get_optimized_geometry()

In [9]:
from numpy import argsort
oname = join(protonated, geom+"-protonated.pdb")

inname = join(tempdir, geom+"-cleaved.pdb")
with open(inname) as ifile:
    sys = read_pdb(ifile)

inname = join(tempdir, geom+"-justligand-babel.pdb")
with open(inname) as ifile:
    lsys = read_pdb(ifile)
lsys_target = list(lsys.keys())[0]

# Remove link hydrogen and realign covalent bond
if "cov" in site:
    def find_nearest(subsys, target, h):
        distlist = []
        for i, at in enumerate(subsys[target]):
            inner = [norm(array(x.get_position()) - array(at.get_position())) for x in sys[link]]
            distlist.append(min(inner))
        minat = argsort(distlist)

        idx = None
        if not h:
            if (subsys[target][minat[0]].sym != 'H'):
                idx = minat[0]
            elif (subsys[target][minat[1]].sym != 'H'):
                idx = minat[1]
        else:
            if (subsys[target][minat[0]].sym == 'H'):
                idx = minat[0]
            elif (subsys[target][minat[1]].sym == 'H'):
                idx = minat[1]
        return idx

    # Delete the linking hydrogen
    idx = find_nearest(lsys, lsys_target, True)
    del lsys[lsys_target][idx]

    # Optimize the hydrogen positions with xtb while constraining other elements.
    tempsys = System()
    tempsys["LIG:0"] = lsys[lsys_target]
    tempsys[link] = sys[link]
    
    optsys = constrained_optimization_xtb(tempsys)

    sys["LIG:0"] = optsys["LIG:0"]
else:
    sys["LIG:0"] = lsys[lsys_target]

if not exists(oname):
    with open(oname, "w") as ofile:
        write_pdb(sys, ofile)

In [10]:
oname1 = join(tempdir, geom+"-script-p.pml")
oname2 = join(picdir, geom+"-pocket-protonated.png")

if not exists(oname2):
    with open(oname1, "w") as ofile:
        ofile.write("load " + join(protonated, geom+"-protonated.pdb") + "\n")
        ofile.write("hide everything\n")
        ofile.write("show licorice sticks, resn LIG\n")
        ofile.write("color yellow, resn LIG\n")
        ofile.write("select near_lig, resn lig around 10\n")
        ofile.write("show lines, near_lig\n")
        ofile.write("color blue, near_lig\n")
        ofile.write("orient resname LIG\n")
        ofile.write("zoom resname LIG, complete=1, buffer=4\n")
        ofile.write("orient resname LIG, complete=1, buffer=4\n")
        ofile.write("bg_color white\n")
        ofile.write("png " + join(oname2) + ", width=10cm, dpi=300, ray=1\n")
        ofile.write("quit\n")
    system("pymol " + oname1)

Optimize.

In [11]:
from BigDFT.XTBInterop import XTBCalculator
code = XTBCalculator(verbose=True)
code.run(sys=sys, name=geom, run_dir=join(protonated, geom), opt=True, gfnff=True, dry_run=True)

Initialize a Calculator with OMP_NUM_THREADS=2 and command xtb
Run directory protonated/Mpro-x0305
Executing command:  xtb Mpro-x0305.tmol --define


In [12]:
optsys = code.get_optimized_geometry()

oname = join(outdir, geom + "-m.pdb")
if not exists(oname):
    with open(oname, "w") as ofile:
        write_pdb(optsys, ofile)

Create a picture.

In [13]:
oname1 = join(tempdir, geom+"-script-p.pml")
oname2 = join(picdir, geom+"-pocket-m.png")

if not exists(oname2):
    with open(oname1, "w") as ofile:
        ofile.write("load " + join(outdir, geom+"-m.pdb") + "\n")
        ofile.write("hide everything\n")
        ofile.write("show licorice sticks, resn LIG\n")
        ofile.write("color yellow, resn LIG\n")
        ofile.write("select near_lig, resn lig around 10\n")
        ofile.write("show lines, near_lig\n")
        ofile.write("color blue, near_lig\n")
        ofile.write("orient resname LIG\n")
        ofile.write("zoom resname LIG, complete=1, buffer=4\n")
        ofile.write("orient resname LIG, complete=1, buffer=4\n")
        ofile.write("bg_color white\n")
        ofile.write("png " + join(oname2) + ", width=10cm, dpi=300, ray=1\n")
        ofile.write("quit\n")
    system("pymol " + oname1)